In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import time
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.svm import SVC

# Градиентный бустинг на деревьях

## Задание 1-3
Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [2]:
X_train = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
X_train.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire'], inplace = True, axis = 1)
a = X_train.count()
z = X_train.shape[0]
for i in range (0,X_train.shape[1]-1,1):
    if (a[i]<z): print(a[i:i+1:1])

first_blood_time    77677
dtype: int64
first_blood_team    77677
dtype: int64
first_blood_player1    77677
dtype: int64
first_blood_player2    53243
dtype: int64
radiant_bottle_time    81539
dtype: int64
radiant_courier_time    96538
dtype: int64
radiant_flying_courier_time    69751
dtype: int64
radiant_first_ward_time    95394
dtype: int64
dire_bottle_time    81087
dtype: int64
dire_courier_time    96554
dtype: int64
dire_flying_courier_time    71132
dtype: int64


## Список неполностью заполненных значений с указанием реальной обстановки/причины:

    first_blood_time               77677   За первые 5 минут не всегда есть первое убийство

    first_blood_team               77677   За первые 5 минут не всегда есть первое убийство=> нет всегда такой команды

    first_blood_player1            77677   За первые 5 минут не всегда есть первое убийство=> нет всегда и такого игрока

    first_blood_player2            53243   За первые 5 минут не всегда есть первое убийство=> а вот помощников еще меньше

    radiant_bottle_time            81539   За первые 5 минут не куплен данный предмет рэдиантами

    radiant_courier_time           96538   За первые 5 минут не куплен данный предмет рэдиантами

    radiant_flying_courier_time    69751   За первые 5 минут не куплен данный предмет рэдиантами

    radiant_first_ward_time        95394   За первые 5 минут не куплен/установлен данный предмет рэдиантами

    dire_bottle_time               81087   За первые 5 минут не куплен данный предмет даирами

    dire_courier_time              96554   За первые 5 минут не куплен данный предмет даирами

    dire_flying_courier_time       71132   За первые 5 минут не куплен данный предмет даирами

    dire_first_ward_time           95404   За первые 5 минут не куплен данный предмет даирами


# Задание 4
Какой столбец содержит целевую переменную? Запишите его название.

Целевая переменная - 'radiant_win'

## Задание 5
Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

### Итог
Классификатор настраивается довольно долго. Время настройки растет как функция от времени. Достигнут лимит качества, так как при увеличении числа деревьев качество меняется довольно незначительно. Скорее всего качество будет расти, но вскоре будет достигнут лимит. (возможно, предел находится в районе 70-72 %)

Время для обучения 10 деревьев  0:01:02.519576; 
0.6645339957137539

Время для обучения 20 деревьев  0:01:56.767678; 
0.6819901320753952

Время для обучения 30 деревьев 0:03:05.319599; 
0.6891167972431018

In [3]:
X_test = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire'], inplace = True, axis = 1)
#scaler = StandardScaler()
#X_test = scaler.fit_transform(X_test)
# Создание выборки для обучения
kf = KFold(n_splits = 5, shuffle=True)
kf.split(X_test, y_test)

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=10)#,max_depth = 3, learning_rate =0.1 )
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 10 деревьев ', datetime.datetime.now() - start_time )
print(a.mean())

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=20,max_depth = 3)#, learning_rate =0.1)
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 20 деревьев ', datetime.datetime.now() - start_time )
print(a.mean())

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=30,max_depth = 3)#, learning_rate =0.1)
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 30 деревьев', datetime.datetime.now() - start_time )
print(a.mean())

Время для обучения 10 деревьев  0:00:51.989710
0.6654458562456946
Время для обучения 20 деревьев  0:01:33.444589
0.6818068818049412
Время для обучения 30 деревьев 0:02:22.481791
0.689971756375909


# Логистическая регрессия

## Задание 1
Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
### Итог
Наилучшее качество получилось равным ~51.3%. Качество градиентного бучтинга значительно лучше (около 68.9%). Данная разница связана с тем, что бустинг на деревьях не зависит от представленных данных, тоесть от масштабирования, что играет существенную разницу для логистической регрессии. (Логистическая регрессия работает в несколько раз быстрее.) для регрессии 0:00:30.510173, для градиентного бустинга на 30 деревьях 0:03:05.319599

При использовании масштабирования признаков качество вырастает до 71.6%. Что чуть больше чем у градиентного бустинга.

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire'], inplace = True, axis = 1)

# Нормировка выборки
#X_test = StandardScaler().fit_transform(X_test)

# Создание выборки для обучения
kf = KFold(n_splits = 5, shuffle=True)
kf.split(X_test, y_test)

#Поиск лучшего значения для параметра
best_value = 0
i_best = 0
start_time = datetime.datetime.now()
for i in (0.01,0.1, 0.2, 0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.1,1.2,1.3,1.4,1.5,2,3,4,5,6,7,8,9,10,15,20,50,100):
    # Просто прогонка логистической регресии
    
    LR = LogisticRegression(penalty='l2',C=i).fit(X_test, y_test)
    a = cross_val_score(LR, X_test, y_test, cv = kf, scoring = 'roc_auc')
    if (a.mean()>best_value):
        best_value = a.mean()
        i_best = i
print( 'Время для простой прогонки', datetime.datetime.now() - start_time )
print(best_value, i_best)

## Задание 2
Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?
### Итог
Качество не улучшилось при удалении этих столбцов, скорее всего содержащаяся там информация используется в плохом качестве

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire','lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 
             'r5_hero','d1_hero', 'd2_hero','d3_hero','d4_hero','d5_hero'],inplace = True, axis = 1)


# Создание выборки для обучения. Масштабироание
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
kf = KFold(n_splits = 5, shuffle=True)
kf.split(X_test, y_test)

#Поиск наилучшего значения для коэффициента логистической регрессии
best_value = 0
i_best = 0
start_time = datetime.datetime.now()
for i in (0.1, 0.2, 0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.1,1.2,1.3,1.4,1.5,2,3,4,5,6,7,8,9,10,15,20):
    LR = LogisticRegression(penalty='l2',C=i).fit(X_test, y_test)
    a = cross_val_score(LR, X_test, y_test, cv = kf, scoring = 'roc_auc')
    if (a.mean()>best_value):
        best_value = a.mean()
        i_best = i
print( 'Время для простой прогонки', datetime.datetime.now() - start_time )
print(best_value, i_best)

## Задание 3.
На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).
### Итог
всего в игре 112 героев

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
X_test['r1_hero'].max()

## Задание 4.
Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
### Итог
Получился массив размером (97230, 204)

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
data = pd.read_csv(r"D:\works\features/features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire', 'lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 
             'r5_hero','d1_hero', 'd2_hero','d3_hero','d4_hero','d5_hero'],inplace = True, axis = 1)

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
# N — количество различных героев в выборке
N = data['d2_hero'].max()
X_pick = np.zeros((data.shape[0], N))


for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test = np.hstack((X_test, X_pick))
X_test.shape

## Задание 5
Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?
### Итог
Качество улучшилось 71.6% против 75.2%. Это объясняется тем, что информация, которая содержится в дополнительных столбцах, теперь используется по прямому назначению (появились более выигрышные и более играющие герои)

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
data   = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire', 'lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 
             'r5_hero','d1_hero', 'd2_hero','d3_hero','d4_hero','d5_hero'],inplace = True, axis = 1)

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
N = data['d2_hero'].max()
X_pick = np.zeros((data.shape[0], N))


for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test = np.hstack((X_test, X_pick))
data = pd.DataFrame()
# Создание выборки для обучения
kf = KFold(n_splits = 5, shuffle=True, random_state = 0)
kf.split(X_test, y_test)

#Поиск наилучшего значения для коэффициента логистической регрессии
best_value = 0
i_best = 0

for i in (0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 3, 5, 7, 10, 20, 50, 100):
    start_time = datetime.datetime.now()
    LR = LogisticRegression(penalty='l2',C=i).fit(X_test, y_test)
    a = cross_val_score(LR, X_test, y_test, cv = kf, scoring = 'roc_auc')
    if (a.mean()>best_value):
        best_value = a.mean()
        i_best = i
    print( 'Время для простой прогонки', datetime.datetime.now() - start_time )
print(best_value, i_best)

## Задание 6.
Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).
### Итог
Максимальное значение для выигрыша 0.996517, для проигрыша 0.008591

In [ ]:
#Чтение тренировочных данных и удаление части "будущего"
X_train = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
data_train = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
X_train.fillna(0, inplace = True)
y_train = X_train['radiant_win']
X_train.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire', 'lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 
             'r5_hero','d1_hero', 'd2_hero','d3_hero','d4_hero','d5_hero'],inplace = True, axis = 1)

#Чтение тестовых данных и удаление столбцов с героями
X_test = pd.read_csv(r"D:\works\features_test\features_test.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
data_test = pd.read_csv(r"D:\works\features_test\features_test.csv", index_col='match_id')
X_test.drop(['lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 'r5_hero','d1_hero', 'd2_hero','d3_hero',
             'd4_hero','d5_hero'],inplace = True, axis = 1)

# Масштабирование даннх
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
kf = KFold(n_splits = 3, shuffle=True)
kf.split(X_train, y_train)

N = data_test['d2_hero'].max()
X_pick_train = np.zeros((data_train.shape[0], N))
X_pick_test = np.zeros((data_test.shape[0], N))

# Добавление героев в тренировочную выборку
for i, match_id in enumerate(data_train.index):
    for p in range(5):
        X_pick_train[i, data_train.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_train[i, data_train.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1      
X_train = np.hstack((X_train, X_pick_train))

# Добавление героев в тестовую выборку
for i, match_id in enumerate(data_test.index):
    for p in range(5):
        X_pick_test[i, data_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, data_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1      
X_test = np.hstack((X_test, X_pick_test))
data_test = pd.DataFrame()

#Обучение и построение предсказание для логистической регрессии
LR = LogisticRegression(penalty='l2',C= 1).fit(X_train, y_train)
a = pd.read_csv(r"D:\works\features_test\features_test.csv", usecols = [0])
a['radiant_win'] = LR.predict_proba(X_test)[:,1]
a.to_csv(r"D:\works\final_test.csv", sep = ' ', header = False, index = False, mode ='w+')
print(a.min(), a.max())

# Случайный лес классификатор 
0.6305000070125029; 
Время для 10 деревьев 0:00:54.525119

0.6528236711683177; 
Время для 20 деревьев 0:01:29.975146

0.669879466254194; 
Время для 30 деревьев 0:01:54.844569

Скорость примерно похожа, но качество ниже

In [ ]:
X_test = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire'], inplace = True, axis = 1)
#scaler = StandardScaler()
#X_test = scaler.fit_transform(X_test)
# Создание выборки для обучения
kf = KFold(n_splits=5, shuffle=True, random_state=1)
kf.split(X_test, y_test)
  
start_time = datetime.datetime.now()    
regr = RandomForestClassifier(n_estimators=10)#,random_state=1)
a = cross_val_score(regr, X_test, y_test, scoring = 'roc_auc', cv = kf)
print(a.mean())
print( 'Время для 10 деревьев', datetime.datetime.now() - start_time )

start_time = datetime.datetime.now()    
regr = RandomForestClassifier(n_estimators=20)#,random_state=1)
a = cross_val_score(regr, X_test, y_test, scoring = 'roc_auc', cv = kf)
print(a.mean())
print( 'Время для 20 деревьев', datetime.datetime.now() - start_time )

start_time = datetime.datetime.now()    
regr = RandomForestClassifier(n_estimators=30)#,random_state=1)
a = cross_val_score(regr, X_test, y_test, scoring = 'roc_auc', cv = kf)
print(a.mean())
print( 'Время для 30 деревьев', datetime.datetime.now() - start_time )

# SGDClassifier
Linear classifiers (SVM, logistic regression, a.o.) with SGD training.

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
data = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire', 'lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 
             'r5_hero','d1_hero', 'd2_hero','d3_hero','d4_hero','d5_hero'],inplace = True, axis = 1)

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
N = data['d2_hero'].max()
X_pick = np.zeros((data.shape[0], N))


for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test = np.hstack((X_test, X_pick))
data = pd.DataFrame()

# Создание выборки для обучения
kf = KFold(n_splits = 5, shuffle=True, random_state = 0)
kf.split(X_test, y_test)

start_time = datetime.datetime.now()  
clf = linear_model.SGDClassifier(penalty='l2', max_iter = 2000)
a = cross_val_score(clf, X_test, y_test, cv = kf, scoring = 'roc_auc')
print(a.mean())
print( 'Время для аппроксимации для SGDClassifier', datetime.datetime.now() - start_time )

#start_time = datetime.datetime.now()  
#clf = linear_model.Perceptron(penalty='l2', max_iter = 100)
#a = cross_val_score(clf, X_test, y_test, cv = kf, scoring = 'roc_auc')
#print(a.mean())
#print( 'Время для аппроксимации для персептрона', datetime.datetime.now() - start_time )

#start_time = datetime.datetime.now()  
#clf = linear_model.LinearSVC(penalty='l2', max_iter = 100)
#a = cross_val_score(clf, X_test, y_test, cv = kf, scoring = 'roc_auc')
#print(a.mean())
#print( 'Время для аппроксимации для SVC', datetime.datetime.now() - start_time )

# Градиентный бустинг на деревьях решений
start_time = datetime.datetime.now()
GBC = GradientBoostingClassifier(n_estimators=30,max_depth = 3)#, learning_rate =0.1)
GBC.fit(X_test, y_test)
a = cross_val_score(GBC, X_test, y_test, cv = kf, scoring = 'roc_auc')
print( 'Время для обучения 30 деревьев', datetime.datetime.now() - start_time )
print(a.mean())

# SVM classification
start_time = datetime.datetime.now()  
clf = SVC(kernel='linear', random_state=241)
a = cross_val_score(clf, X_test, y_test, cv = kf, scoring = 'roc_auc')
print(a.mean())
print( 'Время для аппроксимации для SVM', datetime.datetime.now() - start_time )

 # SVM

In [ ]:
#Чтение данных и удаление части "будущего"
X_test = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
data = pd.read_csv(r"D:\works\features\features.csv", index_col='match_id')
X_test.fillna(0, inplace = True)
y_test = X_test['radiant_win']
X_test.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant',
             'barracks_status_dire', 'lobby_type', 'r1_hero', 'r2_hero','r3_hero', 'r4_hero', 
             'r5_hero','d1_hero', 'd2_hero','d3_hero','d4_hero','d1_hero'],inplace = True, axis = 1)

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
N = data['d2_hero'].max()
X_pick = np.zeros((data.shape[0], N))


for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test = np.hstack((X_test, X_pick))
data = pd.DataFrame()

# Создание выборки для обучения
kf = KFold(n_splits = 5, shuffle=True, random_state = 0)
kf.split(X_test, y_test)

start_time = datetime.datetime.now()  
clf = SVC(kernel='rbf')
a = cross_val_score(clf, X_test, y_test, cv = kf, scoring = 'roc_auc')
print(a.mean())
print( 'Время для аппроксимации для SVM', datetime.datetime.now() - start_time )